In [14]:
# List of TTPs to check
ttp_list = [
    'T1194', 'T1088', 'T1346', 'T1065', 'T1500', 'T1311', 'T1432', 'T1002', 'T1043', 
    'T1433', 'T1329', 'T1312', 'T1179', 'T1517', 'T1093', 'T0872', 'T1079', 'T1009', 
    'T0840', 'T1099', 'T1053.004', 'T1045', 'T1426', 'T1100', 'T1512', 'T1022', 'T1438', 
    'T1420', 'T1532', 'T1508', 'T0871', 'T1533', 'T1476', 'T1077', 'T1191', 'T1402', 
    'T1341', 'T1024', 'T1117', 'T1047.001', 'T1345', 'T1004', 'T1497.004', 'T1064', 
    'T1247', 'T1089', 'T1249', 'T1086', 'T1260', 'T1061', 'T0853', 'T1143', 'T0847', 
    'T1122', 'T1084', 'T1351', 'T1268', 'T0807', 'T1023', 'T1107', 'T1444', 'T1660', 
    'T0831', 'T0809', 'T1081', 'T1101', 'T1437', 'T1076', 'T1503.004', 'T1035', 'T1412', 
    'T1308', 'T1328', 'T1013', 'T1031', 'T1060', 'T1193', 'T1085', 'T0855', 'T1513', 
    'T1116', 'T1168', 'T1094', 'T1447', 'T1138', 'T1063', 'T1183', 'T1032', 'T1158', 
    'T1418', 'T1502', 'T1337', 'T1145', 'T1575', 'T1347', 'T1503.003', 'T1503', 'T1429', 
    'T1544', 'T1050', 'T1108', 'T1073', 'T1015', 'T1038', 'T1192', 'T1362', 'T1066'
]

In [15]:
import json
import requests

def get_attack_stix_data(version_url):
    """
    Fetches STIX data for a given MITRE ATT&CK version.
    
    Args:
        version_url (str): URL for the MITRE ATT&CK STIX data.
    
    Returns:
        dict: Parsed JSON data from the URL.
    """
    response = requests.get(version_url)
    if response.status_code == 200:
        return response.json()
    else:
        raise ValueError(f"Error fetching data from {version_url}. Status code: {response.status_code}")

def find_ttp_in_version(stix_data, ttp_id):
    """
    Checks if a specific TTP ID exists in the STIX data.

    Args:
        stix_data (dict): Parsed STIX JSON data.
        ttp_id (str): The TTP ID to search for.

    Returns:
        bool: True if the TTP is present, False otherwise.
    """
    for obj in stix_data.get('objects', []):
        if obj.get('type') == 'attack-pattern' and ttp_id in obj.get('external_references', [{}])[0].get('external_id', ''):
            return True
    return False

def find_latest_version_for_ttps(ttp_list, versions):
    """
    Finds the latest version where each TTP is present.

    Args:
        ttp_list (list): List of TTP IDs to check.
        versions (list): List of URLs for the STIX files.

    Returns:
        dict: A dictionary with TTP IDs as keys and the latest version they were found in.
    """
    latest_versions = {}

    for ttp_id in ttp_list:
        for version_url in reversed(versions):  # Start from the latest version
            try:
                # Fetch STIX data for the current version
                stix_data = get_attack_stix_data(version_url)
                
                # Check for the TTP in the current version
                if find_ttp_in_version(stix_data, ttp_id):
                    version_name = version_url.split('/')[-1]  # Extract the version name
                    latest_versions[ttp_id] = version_name
                    break  # Stop once the TTP is found in this version
            except Exception as e:
                print(f"Error processing {version_url}: {str(e)}")

    return latest_versions

In [16]:
# List of STIX file versions with raw URLs for each version
versions = [
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-1.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-2.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-3.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-4.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-5.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-6.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-7.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-8.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-9.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-10.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-11.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-12.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-13.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-14.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-15.0.json",
    "https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/enterprise-attack/enterprise-attack-16.0.json"
]

# Check for each TTP across the versions
#ttp_results = check_ttps_across_versions(ttp_list, versions)

# Display the results
#for ttp, versions_found in ttp_results.items():
#    if versions_found:
#        print(f"TTP {ttp} found in versions: {', '.join(versions_found)}")
#    else:
#        print(f"TTP {ttp} not found in any version")

In [17]:
# Find the latest version for each TTP in the list
latest_versions = find_latest_version_for_ttps(ttp_list, versions)

# Display the results
for ttp_id, latest_version in latest_versions.items():
    print(f"Latest version for TTP {ttp_id}: {latest_version}")

Latest version for TTP T1194: enterprise-attack-16.0.json
Latest version for TTP T1088: enterprise-attack-16.0.json
Latest version for TTP T1065: enterprise-attack-16.0.json
Latest version for TTP T1500: enterprise-attack-16.0.json
Latest version for TTP T1002: enterprise-attack-16.0.json
Latest version for TTP T1043: enterprise-attack-16.0.json
Latest version for TTP T1179: enterprise-attack-16.0.json
Latest version for TTP T1093: enterprise-attack-16.0.json
Latest version for TTP T1079: enterprise-attack-16.0.json
Latest version for TTP T1009: enterprise-attack-16.0.json
Latest version for TTP T1099: enterprise-attack-16.0.json
Latest version for TTP T1053.004: enterprise-attack-16.0.json
Latest version for TTP T1045: enterprise-attack-16.0.json
Latest version for TTP T1100: enterprise-attack-16.0.json
Latest version for TTP T1022: enterprise-attack-16.0.json
Latest version for TTP T1077: enterprise-attack-16.0.json
Latest version for TTP T1191: enterprise-attack-16.0.json
Latest ver